In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())

True
0


In [ ]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from collections import defaultdict


In [4]:
import os
import cv2
import torch
import pandas as pd
from PIL import Image
from facenet_pytorch import MTCNN

# Initialize MTCNN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mtcnn = MTCNN(keep_all=False, device=device)

# Directories
IMG_AFFECT_DATA_DIR = "./AffectNetUnzip/"
output_dir = "./processed_data/"
os.makedirs(output_dir, exist_ok=True)

# Subdirectories for saving processed files
aligned_images_dir = os.path.join(output_dir, "aligned_images/")
landmarked_images_dir = os.path.join(output_dir, "landmarked_images/")
tensor_dir = os.path.join(output_dir, "tensors/")
metadata_file = os.path.join(output_dir, "metadata.csv")

os.makedirs(aligned_images_dir, exist_ok=True)
os.makedirs(landmarked_images_dir, exist_ok=True)
os.makedirs(tensor_dir, exist_ok=True)

# Splits and classes
splits = ["train", "val", "test"]
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8']

# Metadata storage
metadata = []

def align_face(image):
    """Align face using MTCNN landmarks."""
    image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert to PIL format
    boxes, probs, landmarks = mtcnn.detect(image_pil, landmarks=True)
    if landmarks is not None:
        # Extract eye coordinates
        left_eye = landmarks[0][0]
        right_eye = landmarks[0][1]
        dx = right_eye[0] - left_eye[0]
        dy = right_eye[1] - left_eye[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Rotate image
        center = (image.shape[1] // 2, image.shape[0] // 2)
        rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
        aligned_image = cv2.warpAffine(image, rot_mat, (image.shape[1], image.shape[0]))
        return aligned_image, landmarks[0]
    return image, None

# Additional list to track unrecognized images
unrecognized_images = []

# Process images
for split in splits:
    split_dir = os.path.join(IMG_AFFECT_DATA_DIR, split)
    for cls in classes:
        class_dir = os.path.join(split_dir, cls)
        if os.path.isdir(class_dir):
            images = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
            for img_path in images:
                img = cv2.imread(img_path)
                if img is None:
                    continue
                
                # Attempt to align face
                aligned_img, landmarks = align_face(img)
                
                if aligned_img is None or landmarks is None:
                    # Log unrecognized images
                    unrecognized_images.append({
                        "split": split,
                        "class": cls,
                        "path": img_path
                    })
                    continue  # Skip to the next image
                
                # Save aligned image
                aligned_save_path = os.path.join(aligned_images_dir, f"{split}_{cls}_{os.path.basename(img_path)}")
                Image.fromarray(cv2.cvtColor(aligned_img, cv2.COLOR_BGR2RGB)).save(aligned_save_path)
                
                # Save image with landmarks
                if landmarks is not None:
                    for (x, y) in landmarks:
                        cv2.circle(aligned_img, (int(x), int(y)), 5, (0, 255, 0), -1)
                    landmarked_save_path = os.path.join(landmarked_images_dir, f"{split}_{cls}_landmarks_{os.path.basename(img_path)}")
                    Image.fromarray(cv2.cvtColor(aligned_img, cv2.COLOR_BGR2RGB)).save(landmarked_save_path)
                
                # Save tensor
                tensor_save_path = os.path.join(tensor_dir, f"{split}_{cls}_{os.path.splitext(os.path.basename(img_path))[0]}.pt")
                aligned_tensor = torch.from_numpy(cv2.cvtColor(aligned_img, cv2.COLOR_BGR2RGB)).permute(2, 0, 1)
                torch.save(aligned_tensor, tensor_save_path)
                
                # Store metadata
                metadata.append({
                    "split": split,
                    "class": cls,
                    "original_path": img_path,
                    "aligned_path": aligned_save_path,
                    "landmarked_path": landmarked_save_path if landmarks is not None else None,
                    "tensor_path": tensor_save_path,
                    "landmarks": landmarks.tolist(),
                    "is_recognized": True
                })

# Save metadata to CSV
metadata_df = pd.DataFrame(metadata)
metadata_df.to_csv(metadata_file, index=False)

# Save unrecognized images to a separate CSV
if unrecognized_images:
    unrecognized_df = pd.DataFrame(unrecognized_images)
    unrecognized_df.to_csv(os.path.join(output_dir, "unrecognized_images.csv"), index=False)

print("All processing complete!")
if unrecognized_images:
    print(f"{len(unrecognized_images)} images could not be recognized. See 'unrecognized_images.csv' for details.")






All processing complete!
158 images could not be recognized. See 'unrecognized_images.csv' for details.
